# Data Analysis with Spark Sql
this notebook aims to provide a deeper understanding of the processed data, to give context to the upcomming visualizations

### Spark Session
establish a connector between MongoDB and Spark to create Spark Tables

In [154]:
# imports
from config import MONGO_HOST_REMOTE, MONGO_DB_REMOTE
import pandas as pd
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
print(f"pyspark version: {pyspark.__version__}")

pyspark version: 3.3.2


In [159]:
# set configuration to install java packages of mongo-spark connector
spark_conf = pyspark.SparkConf().set("spark.jars.packages",
                              "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
                            .setAppName("My App")


In [160]:

# start spark session
spark = SparkSession.builder \
    .config(conf=spark_conf) \
    .getOrCreate()

In [161]:
# Create uri for host and db to access mongo
uri =  f"{MONGO_HOST_REMOTE}/{MONGO_DB_REMOTE}"


collections = {
    "players": "raw_players",
    "gdp": "world_bank_gdp",
    "countries": "countries"
}
# turn every collection into a spark table
for table, collection in collections.items():
    df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", uri + "." + collection).load()
    df.createOrReplaceTempView(table)
    

# Analysis
all collections have been turned into spark tables. Now we can start analysing

### Analyze players data

In [46]:
def show_headers(data: list):
    return pd.DataFrame(columns=["Attributes"], data=data)
    

In [47]:
query = """SELECT * FROM players LIMIT 0"""
df = spark.sql(query)
pd_df = show_headers(df.columns)
pd_df

,Attributes
0,_id
1,assists
2,balls_recoverd
3,clearance_attempted
4,club
5,cross_accuracy
6,cross_attempted
7,cross_complted
8,distance_covered
9,fouls_committed


### Top 10 Rows from Table "players"

In [51]:
# let the analysis begin
query = """SELECT * FROM players LIMIT 10"""
df = spark.sql(query)
pandas_df = df.toPandas()
pandas_df

,_id,assists,balls_recoverd,clearance_attempted,club,cross_accuracy,cross_attempted,cross_complted,distance_covered,fouls_committed,...,pass_attempted,pass_completed,player_name,position,red,serial,t_lost,t_won,tackles,yellow
0,"(648eddf3583fa9e84fb89e73,)",0,NaN,NaN,Real Madrid,0,0,0,64.2,NaN,...,483,365,Courtois,Goalkeeper,NaN,447,NaN,NaN,NaN,NaN
1,"(648eddf3583fa9e84fb89e74,)",6,29.0,0.0,Real Madrid,31,19,6,133.0,13.0,...,451,377,Vinícius Júnior,Forward,1.0,121,8.0,3.0,11.0,0.0
2,"(648eddf3583fa9e84fb89e75,)",1,6.0,10.0,Real Madrid,20,10,2,121.5,2.0,...,523,433,Benzema,Forward,1.0,428,0.0,1.0,1.0,0.0
3,"(648eddf3583fa9e84fb89e76,)",4,39.0,4.0,Real Madrid,25,12,3,124.5,3.0,...,799,723,Modrić,Midfielder,1.0,54,6.0,3.0,9.0,0.0
4,"(648eddf3583fa9e84fb89e77,)",0,76.0,32.0,Real Madrid,0,2,0,110.4,17.0,...,643,563,Éder Militão,Defender,4.0,1,13.0,6.0,19.0,0.0
5,"(648eddf3583fa9e84fb89e78,)",0,49.0,28.0,Real Madrid,11,17,2,112.3,9.0,...,657,606,Alaba,Defender,1.0,22,6.0,12.0,18.0,0.0
6,"(648eddf3583fa9e84fb89e79,)",1,50.0,30.0,Real Madrid,34,23,8,112.8,15.0,...,576,503,Carvajal,Defender,3.0,21,6.0,6.0,12.0,0.0
7,"(648eddf3583fa9e84fb89e7a,)",0,76.0,19.0,Real Madrid,25,4,1,107.6,17.0,...,538,482,Casemiro,Midfielder,3.0,1,18.0,13.0,31.0,0.0
8,"(648eddf3583fa9e84fb89e7b,)",0,37.0,4.0,Real Madrid,20,34,7,116.5,7.0,...,814,752,Kroos,Midfielder,0.0,64,6.0,5.0,11.0,0.0
9,"(648eddf3583fa9e84fb89e7c,)",2,36.0,15.0,Real Madrid,0,0,0,96.3,7.0,...,208,173,Mendy,Defender,3.0,70,1.0,5.0,6.0,0.0


### Average fouls per position

In [55]:
query = """SELECT position, AVG(fouls_committed) as average_fouls_committed FROM players GROUP BY position ORDER BY average_fouls_committed DESC;"""
df = spark.sql(query)
pandas_df = df.toPandas()
pandas_df

,position,average_fouls_committed
0,Midfielder,5.521739
1,Defender,4.617925
2,Forward,4.423077
3,Goalkeeper,1.000000


### Average fouls per nationality
we ignore nationalities without any foul information

In [172]:
query = """SELECT nationality, AVG(fouls_committed) as average_fouls_committed FROM players
                WHERE fouls_committed is not NULL
            GROUP BY nationality ORDER BY average_fouls_committed DESC;
            """
df = spark.sql(query)
pandas_df = df.toPandas()
pandas_df


,nationality,average_fouls_committed
0,Mozambique,15.000000
1,Austria,9.500000
2,Colombia,8.200000
3,North Macedonia,8.000000
4,Angola,7.000000
...,...,...
64,Russia,1.857143
65,Kosovo,1.000000
66,Finland,1.000000
67,Venezuela,1.000000


## Analyse Table countries

### Violent Crimes per country per 100 000 citizens

In [170]:
query = """SELECT country, count_p_100k FROM countries order by count_p_100k desc"""
df = spark.sql(query)
pandas_df = df.toPandas()
pandas_df

,country,count_p_100k
0,U.S. Virgin Islands,49.3
1,Jamaica,44.7
2,Lesotho,43.6
3,Trinidad and Tobago,38.6
4,El Salvador,37.2
...,...,...
190,Holy See,0.0
191,Isle of Man,0.0
192,Monaco,0.0
193,Saint Helena,0.0


### Analyse GDP Data

In [152]:
query = """SELECT * from gdp order by gdp_eur DESC LIMIT 10"""
df = spark.sql(query)
pandas_df = df.toPandas()
pandas_df.head(10)

,_id,country,countryiso3code,date,decimal,gdp_eur,indicator,obs_status,value
0,"(64957bf1e68bcd31216ca85a,)","(1W, World)",WLD,2021,0,8.867009e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,9.652743e+13
1,"(64957bf0e68bcd31216ca83b,)","(XD, High income)",,2021,0,5.495954e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,5.982967e+13
2,"(64957bf1e68bcd31216ca84e,)","(OE, OECD members)",OED,2021,0,5.352531e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,5.826835e+13
3,"(64957bf1e68bcd31216ca851,)","(V4, Post-demographic dividend)",PST,2021,0,5.049795e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,5.497273e+13
4,"(64957bf0e68bcd31216ca83d,)","(ZT, IDA & IBRD total)",IBT,2021,0,3.496453e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,3.806285e+13
5,"(64957bf1e68bcd31216ca846,)","(XO, Low & middle income)",LMY,2021,0,3.341003e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,3.637059e+13
6,"(64957bf1e68bcd31216ca84c,)","(XP, Middle income)",MIC,2021,0,3.292735e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,3.584514e+13
7,"(64957bf0e68bcd31216ca83c,)","(XF, IBRD only)",IBD,2021,0,3.253926e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,3.542267e+13
8,"(64957bf0e68bcd31216ca831,)","(Z4, East Asia & Pacific)",EAS,2021,0,2.839548e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,3.091169e+13
9,"(64957bf1e68bcd31216ca859,)","(XT, Upper middle income)",,2021,0,2.489781e+13,"(NY.GDP.MKTP.CD, GDP (current US$))",,2.710409e+13


### Join table countries and table gdp based on country value

In [175]:
query = """SELECT c.country, c.count_p_100k, cast((g.gdp_eur / 1000000000) as INT) as billion_gdp_euro
                FROM countries c
                JOIN gdp g ON c.country = TRIM(TRIM('}',SUBSTRING(cast(g.country as String), 5)))
                order by c.count_p_100k desc
            """

df = spark.sql(query)
pandas_df = df.toPandas()
pandas_df.head(100)

,country,count_p_100k,billion_gdp_euro
0,Jamaica,44.7,13
1,Lesotho,43.6,2
2,Trinidad and Tobago,38.6,22
3,El Salvador,37.2,26
4,Honduras,36.3,26
...,...,...,...
95,Sierra Leone,1.7,3
96,Finland,1.6,273
97,Malta,1.6,15
98,Iceland,1.5,23
